In [15]:
import  zhipuai
from zhipuai import ZhipuAI
from PIL import Image
import requests
import matplotlib.pyplot as plt
import numpy as np
import io
from datetime import datetime
import random
import asyncio
import json

In [16]:
# 输入API key
client = ZhipuAI(api_key='这里输入你的智谱AI的API key')

# 游戏日志
game_log = []

# prompt库
Prompt_MonsterBuild='接下来我会给出一个虚构的怪物的名字，他们将在一个虚构的竞技场里对战。首先，需要你为这个怪物编写一段人物小传，并参考宝可梦游戏，编造出符合其名字的3个技能，并做简单的解释。'

# 文件路径
playerA_path = 'playerA.txt'
playerB_path = 'playerB.txt'
log_path = 'game_log.txt'

In [17]:
class Model:
    def __init__(self):
        self.client=ZhipuAI(api_key='这里输入你的智谱AI的API key')

    def generator_model(self, prompt):
        response = self.client.chat.completions.create(
            model="glm-4",
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            top_p=0.7,
            temperature=0.95,
            max_tokens=1024,
            stream=True,
        )

        content_text=''
        for trunk in response:
            content_text =content_text + trunk.choices[0].delta.content
        return content_text

    def referee_model(self, MonsterA, MonsterB, movement, skillA, skillB, skill2use):
        # 手册：https://open.bigmodel.cn/dev/api#characterglm
        movement_letter = 'A' if movement == MonsterA else 'B'
        response = client.chat.completions.create(
            model="charglm-3",  
            meta= {
                "bot_info": "你是一场虚构宝可梦对战的游戏裁判兼旁白，你的任务是根据双方宝可梦的特性描述出某个宝可梦使用某个技能后产生的效果，注意需要考虑双方宝可梦的特性，以及技能的描述和特性，并给出一个合理且详细的战斗描述。",
                "bot_name": "游戏旁白",
                "user_info": "我是一个宝可梦训练师，正在和对手战斗，我会向bot（游戏旁白）提供双方宝可梦的信息。",
                "user_name": "宝可梦训练师",
            },
            messages= [
                {
                    "role": "user",
                    "content": "宝可梦A的名字：" + MonsterA.name
                },
                {
                    "role": "user",
                    "content": "宝可梦A的描述：" + MonsterA.description[0]
                },
                {
                    "role": "user",
                    "content": "宝可梦A的技能：" + skillA
                },
                {
                    "role": "user",
                    "content": "宝可梦B的名字：" + MonsterB.name
                },
                {
                    "role": "user",
                    "content": "宝可梦B的描述：" + MonsterB.description[0]
                },
                {
                    "role": "user",
                    "content": "宝可梦B的技能：" + skillB
                },
                {
                    "role": "user",
                    "content": f"宝可梦{movement_letter}使用技能" + movement.skill_name[skill2use] + "对宝可梦B"
                },

            ]
        )
        return response.choices[0].message.content
    
    def damage_model(self, MonsterA, MonsterB, movement, skillA, skill2use, prompt):
        # 手册：https://open.bigmodel.cn/dev/api#characterglm
        movement_letter = 'A' if movement == MonsterA else 'B'
        response = client.chat.completions.create(
            model="charglm-3",  
            meta= {
                "bot_info": "你是一场虚构宝可梦对战的游戏裁判，你的任务是根据user提供的双方宝可梦的特性，并根据战斗过程的描述，回答本次进攻是优势还是劣势，回答时只回答“优势”两个字或者“劣势”两个字。",
                "bot_name": "游戏裁判",
                "user_info": "我是游戏裁判的助手，负责提供判决相关的信息。",
                "user_name": "助手",
            },
            messages= [
                {
                    "role": "user",
                    "content": "宝可梦A的名字：" + MonsterA.name
                },
                {
                    "role": "user",
                    "content": "宝可梦A的描述：" + MonsterA.description[0]
                },
                {
                    "role": "user",
                    "content": "宝可梦A的技能：" + skillA
                },
                {
                    "role": "user",
                    "content": "宝可梦B的名字：" + MonsterB.name
                },
                {
                    "role": "user",
                    "content": "宝可梦B的描述：" + MonsterB.description[0]
                },

                {
                    "role": "user",
                    "content": f"宝可梦{movement_letter}使用技能" + movement.skill_name[skill2use] + f"战斗过程：{prompt}，请给出回答"
                },
            ]
        )
        return response.choices[0].message.content

    def image_model(self, question, style =''):
        response = self.client.images.generations(
            model="cogview-3", #填写需要调用的模型名称
            prompt=style + question,
        )
        image_url = response.data[0].url
        response = requests.get(image_url)
        img = np.array(Image.open(io.BytesIO(response.content)))    
        return img
    
    def plot_image(img):
        # 显示图片
        plt.imshow(img)
        plt.axis('off')  # 不显示坐标轴
        plt.show()

In [18]:
class GameLog:
    def __init__(self):
        global game_log
    
    def update_log(self, content, owner='system'):
        # 生成时间戳
        now = datetime.now()
        current_time = str(now)
        first_period_index = current_time.find('.')
        time_log = f'[{current_time[:first_period_index]}]: '
        game_log.append(time_log + '[' + owner + ']: ' +content)
    
    def clear_log(self):
        game_log.clear()

    def print_log(self):
        for log in game_log:
            print(log)

    def generate_log(self):
        # 打开一个文件用于写入，如果文件不存在将会被创建
        with open(log_path, "w") as file:
            # 遍历列表中的每个字符串
            for s in game_log:
                # 将字符串写入文件的一行，并添加换行符
                file.write(s + "\n")

In [19]:
class MonsterInfo:
    def __init__(self, filename, logupdate = True, is_test = False):
        global logloader
        global model

        self.filename = filename
        self.player = ''
        self.name = ''
        self.image = []
        self.keyword = []
        self.description = []
        self.skill_name = []
        self.skill_description = []
        self.skill_icon = []
        self.MaxHp = 100
        self.Hp = self.MaxHp

        #测试信息
        self.is_test = is_test
        self.logupdate = logupdate
        
    def inital_generate(self):
        # 赋值宝可梦初始信息
        filename = self.filename
        with open(filename, "r") as file:
            self.name = file.readline().strip()
            self.description.append(file.readline().strip())
            self.player = filename[:filename.find('.')]
        # 如果没填写小传
        if self.description[0] == '':
            description = model.generator_model('接下来我会给出一个虚构的怪物的名字，它可能是一个完全虚构的生物也可能是一个日常生活中真实存在的物品，它将在一个虚构的竞技场里和其他怪物对战,需要你为这个怪物编写一段人物小传。怪物的名字是：'+ self.name)
            description = description.replace(" ", "").replace("\n", "")
            index = description.find('人物小传')
            if index != -1:
                description = description[index+5:]
            self.description[0] = description
            # 上传日志
            logloader.update_log(f'检测到{self.player}没有写描述，已自动生成描述。')
            with open(self.player+'.txt', "w") as file:
                file.write(self.name + '\n')
                file.write(self.description[0] )

        # if self.logupdate:
        logloader.update_log(f'{self.player} 玩家信息已录入')
        logloader.update_log(f'{self.player} 宝可梦名字：{self.name}已录入')
        logloader.update_log(f'{self.player} 宝可梦初始描述已录入')
    
    def image_generate(self):
        # 生成宝可梦图片
        original_prompt = "I now need to generate AI paintings through keywords. Please transform the following content into an English prompt, separated by commas in English, and only answer with the prompt without any additional sentences: "
        for i in range(len(self.description)):
            original_prompt = original_prompt + self.description[i] +' '
        image_prompt = model.generator_model(original_prompt)
        keywords = image_prompt.split(',')
        self.keyword.append(keywords)

        if self.is_test:
            img = np.random.rand(100, 100, 3)  
        else:
            img = model.image_model(image_prompt,style ='single character, only one character, CG')
        
        self.image.append(img)
        if self.logupdate:
            logloader.update_log(f'{self.player}\{self.player}_monster.jpeg'f' 宝可梦图片已录入')
        plt.imsave(f'{self.player}\{self.player}_image{len(self.image)}.jpeg', img, format='jpeg')
        return img
    
    def skill_generate(self):
        # 生成宝可梦技能名称和描述
        description =''
        for i in range(len(self.description)):
            description = description + self.description[i] +' '
        original_prompt = f"""我现在需要虚构一个宝可梦，它可能是一个完全虚构的生物也可能是一个日常生活中真实存在的物品，现在我将提供这个宝可梦的名字和有关描述，请结合其特性并参考宝可梦游戏生成它的3个技能以及相关描述。
                            保证格式为 [技能名字]:技能描述
                            回答过程注意不需要回答额外语句。
                            宝可梦名字：{self.name}
                            宝可梦描述：{description}"""
        skill = model.generator_model(original_prompt)
        with open(f"{self.player}\{self.player}_skill.txt", "w") as file:
                file.write(skill)

        # 字符串切割
        text = skill
        items = text.split('\n\n')
        for item in items:
            # 再次分割每个项，得到属性名和描述
            parts = item.split(':')
            # 提取属性名并添加到name列表
            attribute = parts[0].strip()
            self.skill_name.append(attribute)
            # 提取描述并添加到description列表
            description = ':'.join(parts[1:]).strip()
            self.skill_description.append(description)
            
        if self.is_test == False:
        # 生成宝可梦技能图片
            for i in range(len(self.skill_name)):
                original_prompt = f"""现在我需要生成一张AI绘画，请生成一张技能名为{self.skill_name[i]}的技能图标，技能描述为：{self.skill_description[i]}，请生成英文的prompt，用英文逗号隔开，回答时只回答prompt，不需要额外用语"""
                image_prompt = model.generator_model(original_prompt)
                skill_icon = model.image_model(image_prompt, style = 'skill icon')
                self.skill_icon.append(skill_icon)
                if self.logupdate:
                    logloader.update_log(f'{self.player} 生成了第{i+1}个技能图片（{self.player}_skill_icon{i+1}.jpeg）')
                plt.imsave(f'{self.player}\{self.player}_skill_icon{i+1}.jpeg', skill_icon, format='jpeg')
        return skill
        

<string>:64: SyntaxWarning: invalid escape sequence '\{'
<string>:65: SyntaxWarning: invalid escape sequence '\{'
<string>:79: SyntaxWarning: invalid escape sequence '\{'
<string>:104: SyntaxWarning: invalid escape sequence '\{'
<>:64: SyntaxWarning: invalid escape sequence '\{'
<>:65: SyntaxWarning: invalid escape sequence '\{'
<>:79: SyntaxWarning: invalid escape sequence '\{'
<>:104: SyntaxWarning: invalid escape sequence '\{'
<string>:64: SyntaxWarning: invalid escape sequence '\{'
<string>:65: SyntaxWarning: invalid escape sequence '\{'
<string>:79: SyntaxWarning: invalid escape sequence '\{'
<string>:104: SyntaxWarning: invalid escape sequence '\{'
<>:64: SyntaxWarning: invalid escape sequence '\{'
<>:65: SyntaxWarning: invalid escape sequence '\{'
<>:79: SyntaxWarning: invalid escape sequence '\{'
<>:104: SyntaxWarning: invalid escape sequence '\{'
C:\Users\开摆！\AppData\Local\Temp\ipykernel_11056\3174125214.py:64: SyntaxWarning: invalid escape sequence '\{'
  logloader.update_log

In [20]:
class GameLogic:
    def __init__(self):
        global logloader
        global model

    def info_init(self, filename, skill_return = True):
        # 输入描述
        Monster = MonsterInfo(filename=filename)
        # 怪兽初始化
        Monster.inital_generate()
        #生成怪物形象
        Monster.image_generate()
        # 生成技能
        skill = Monster.skill_generate()

        if skill_return:
            return Monster, skill
        else:
            return Monster
    
    def drop_coin (self):
        # 决定先手
        if random.random() > 0.5:
            return 'A'
        else:
            return 'B'
    
    def end_of_game(self, Monster):
        # 游戏结束
        if Monster.Hp <= 0:
            return True
        else:
            return False
        
    def battle_logic(self, MonsterA, MonsterB, skillA, skillB):
        flag_EOG = False
        logloader.update_log(f'游戏开始初始化')
        # 判断先手
        drop_coin = self.drop_coin()
        logloader.update_log(f'player{drop_coin}先手')

        # 先手执行
        movement = MonsterA if drop_coin == 'A' else MonsterB
        enemy = MonsterB if movement == MonsterA else MonsterA
        # 进入循环
        while flag_EOG == False:
            # print('技能列表：')
            # for i in range(len(movement.skill_name)):
            #     print(f'{movement.skill_name[i]}:{movement.skill_description[i]}')

            print("释放技能")

            # skill2use = input('选择要释放的技能')
            skill2use = random.randint(0,len(movement.skill_name)-1)
            # 检测是否有效
            try:
                skill2use = int(skill2use)
                if skill2use < 0 or skill2use > len(movement.skill_name):
                    raise ValueError
                logloader.update_log(f'{movement.player} 释放了技能：{movement.skill_name[skill2use]}', movement.player)
            except ValueError:
                print('输入的不是有效的整数，请重新输入。')
                continue
            
            result = model.referee_model(MonsterA, MonsterB, movement, skillA, skillB, skill2use)
            logloader.update_log(f'{result}',movement.player)

            print('\n描述：'+result)

            damage_type = model.damage_model(MonsterA, MonsterB, movement, skillA, skill2use, result)

            if damage_type.find('优势')!= -1:
                damage_type = '优势'
                damage = random.randint(20,40)
            elif damage_type.find('劣势')!= -1:
                damage_type = '劣势'
                damage = random.randint(1,15)
            else:
                damage_type = 'None'
                damage = random.randint(1,60)
            logloader.update_log(f'{enemy.name} 受到{damage}点伤害',enemy.player)

            print(f'{enemy.name} 受到{damage}点伤害',damage_type)

            enemy.Hp -= damage
            flag_EOG = self.end_of_game(enemy)

            print(f'{enemy.name} 剩余血量：{enemy.Hp}\n')
            print(f'{movement.name} 剩余血量：{movement.Hp}')

            if flag_EOG == False:
                logloader.update_log(f'{enemy.name} 剩余血量：{enemy.Hp}',enemy.player)
                movement = enemy
                enemy = MonsterB if movement == MonsterA else MonsterA
            else:
                logloader.update_log(f'{movement.name} 获胜')
                winner = movement.player
            print('=========================================================')
        return winner

In [21]:
# 日志初始化
logloader = GameLog()
logloader.clear_log()
logloader.update_log('游戏开始')

# 模型初始化
model = Model()
# 构建游戏逻辑实例
gamelogic = GameLogic()

# 怪物生成
MonsterA, skillA = gamelogic.info_init(playerA_path)
MonsterB, skillB = gamelogic.info_init(playerB_path)

# 场地生产
pass

# 战斗开始
print (gamelogic.battle_logic(MonsterA, MonsterB, skillA, skillB)+' win!')
logloader.generate_log()

APIRequestFailedError: Error code: 400, with error text {"error":{"code":"1301","message":"系统检测到输入或生成内容可能包含不安全或敏感内容，请您避免输入易产生敏感内容的提示语，感谢您的配合。"}}

In [22]:
logloader.generate_log()